In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


# Data analysis

### Imports

In [9]:
# Import list

import numpy as np
import re
import csv

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelBinarizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Read Data 

In [10]:
# Read from path 

# Different Editors paths
PATH_A = "data\sherlock-holm.es_stories_plain-text_advs.txt"
PATH_E = ""
PATH_G = "0. Projects/3/Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"
PATH_J = ""
PATH_M = "Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"

PATHS = [PATH_A, PATH_E, PATH_G, PATH_J, PATH_M]

text = ""

for path in PATHS:
    try:
        # Read the text file
        with open(path, 'r', encoding='utf-8') as file:
            text = file.read()
            print(path)
    except:
        continue
    else:
        break

print(text[:200])

Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt




                        THE ADVENTURES OF SHERLOCK HOLMES

                               Arthur Conan Doyle



                                Table of contents

               A Scandal in Bohem


## Dataset Analysis

In [11]:

nlp = spacy.load("en_core_web_sm")


def categorize_words(text):

  doc = nlp(text)
  word_counts = Counter()
  for token in doc:
    word_counts[token.pos_] += 1
  return word_counts


def plot_word_counts(word_counts):

  labels = list(word_counts.keys())
  counts = list(word_counts.values())
  plt.bar(labels, counts)
  plt.xlabel("Part-of-Speech Tag (POS)")
  plt.ylabel("Word Count")
  plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for better readability
  plt.title("Word Category Distribution")
  plt.tight_layout()
  plt.show()


with open("Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt", "r", encoding="utf-8") as f:
  book_text = f.read()

word_counts = categorize_words(book_text)

plot_word_counts(word_counts)

NameError: name 'spacy' is not defined

### Flesch-Kincaid Level

In [15]:
def calculate_flesch_kincaid(text):

  sentences = text.split(".")
  sentences = [sentence.strip() for sentence in sentences if sentence]  # Remove empty sentences
  words = text.split()

  num_words = len(words)
  num_sentences = len(sentences)

  # Calculate Flesch-Kincaid factors
  avg_words_per_sentence = num_words / num_sentences if num_sentences > 0 else 0
  avg_syllables_per_word = count_syllables(words) / num_words if num_words > 0 else 0

  # Calculate Flesch-Kincaid Grade Level
  flesch_kincaid_score = 206.835 - 1.015 * (avg_words_per_sentence) - 84.6 * (avg_syllables_per_word)

  return flesch_kincaid_score

def count_syllables(words):

  num_syllables = 0
  for word in words:
    vowels = "aeiouAEIOU"
    count = 0
    is_previous_vowel = False
    for char in word:
      if char in vowels:
        if not is_previous_vowel:
          count += 1
        is_previous_vowel = True
      else:
        is_previous_vowel = False
    if word.endswith("e") and not word[-2] in vowels:
      count += 1
    num_syllables += count
  return num_syllables
  
flesch_kincaid_score = calculate_flesch_kincaid(text)
print("Flesch-Kincaid Grade Level:", flesch_kincaid_score)

Flesch-Kincaid Grade Level: 56.06935556492007


### Divide the set

In [ ]:
### Try with different separators in split

# for line in text.split('.' and '?' and '!' and ':' and ';'):
# for line in text.split'\n'):
# for line in text.split('.'):
#     print(tokenizer.texts_(to_sequences([line])[0])


### Try with all replace

# text = text.replace('\n', ' ').replace('!', '.').replace(':', '.').replace('+', ' ')
# text

In [ ]:
# Try with regular expressions

DIVIDERS_ORIGINAL = "\n"
DIVIDERS_ALL = "[,.!?:;\"]|\n\n|--| and | that | which "
DIVIDERS_MIN = "[.!]|\n\n"
DIVIDERS_BAL = "[,.!?]|\n\n|--"
divide_set = False

text_try = text

if divide_set:
    # Delete cover of book and extra information
    text_try = text[980:-550]

# Split following the dividers given
text_try = re.split(DIVIDERS_ORIGINAL, text_try)

# Delete all the new line comments 
text_try = [el.replace('\n', '') for el in text_try]

text_try

['',
 '',
 '',
 '',
 '                        the adventures of sherlock holmes',
 '',
 '                               arthur conan doyle',
 '',
 '',
 '',
 '                                table of contents',
 '',
 '               a scandal in bohemia',
 '               the red-headed league',
 '               a case of identity',
 '               the boscombe valley mystery',
 '               the five orange pips',
 '               the man with the twisted lip',
 '               the adventure of the blue carbuncle',
 '               the adventure of the speckled band',
 "               the adventure of the engineer's thumb",
 '               the adventure of the noble bachelor',
 '               the adventure of the beryl coronet',
 '               the adventure of the copper beeches',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                              a scandal in bohemia',
 '',
 '',
 '',
 '',
 '',
 '                                table of contents',
 '               

### Tokenization

In [ ]:
# Create Tokenizer object in python

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts([text])
# total_words = len(tokenizer.word_index) + 1

tokens = word_tokenize(text)
vocabulary = set(tokens)
total_words = len(vocabulary) + 1

word_to_idx = {word:idx for idx, word in enumerate(vocabulary)}

print(f"total_words: {total_words}")
print("Índice de palabras:", word_to_idx)

In [ ]:
# Create input-output sequences

input_sequences = []
# for line in text_try:
#     token_list = tokenizer.texts_to_sequences([line])[0]
#     for i in range(1, len(token_list)):
#         n_gram_sequence = token_list[:i+1]
#         input_sequences.append(n_gram_sequence)

input_sequences = []
for line in text_try:
    line_list = line.rstrip(",.;:").split(' ')

    token_list = []
    for char in line_list:
        if char in word_to_idx.keys():
            token_list.append(word_to_idx[char])

    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

In [ ]:
# Get the max value to add padding to other entries

average = 0
for seq in input_sequences:
    average += len(seq) 

max_sequence_len, value = max([(len(seq), seq) for seq in input_sequences])

# max_sequence_len = max([len(seq) for seq in input_sequences])
# input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_seq_pad = np.array([np.pad(seq, (max_sequence_len - len(seq), 0), mode='constant') for seq in input_sequences])

print (f"average = {average / len(input_sequences)}")
input_seq_pad

In [ ]:
# Get index of the longest line

# token_list = []
# for line in text_try:
#     token_list.append(tokenizer.texts_to_sequences([line])[0])

# token_list.index(value)


# Print longest line

# for el in value:
#     for word, index in tokenizer.word_index.items():
#         if index == el:
#             print(word)
#             break
# len(input_sequences)
# input_sequences

### Separate input and output

In [ ]:
X = input_seq_pad[:, :-1]
y = input_seq_pad[:, -1]

X

In [ ]:
y

### Data analysis

In [83]:
words = []
index_word = {}


for sentence in text_try:
    sentence_w = sentence.rstrip('\",.;:!?')
    sentence_w = sentence_w.replace(",","").replace(".","")
    sentence_w = sentence_w.replace(";","").replace(":","")
    sentence_w = sentence_w.replace("!","").replace("?","")
    sentence_w = sentence_w.replace("\"","").replace("'","")
    sentence_w = sentence_w.replace("(","").replace(")","")
    sentence_w = sentence_w.replace("--"," ")

    sentence_w = sentence_w.split(" ")
    for word in sentence_w:
        if word != "":
            if word not in index_word.keys():
                index_word[word] = len(words)
                words.append([word, 1])
            else:
                words[index_word[word]][1] +=1
            
#words_np = np.array(words)

words_sort = sorted(words, key=lambda x: x[1])
words_sort.reverse()

col1 = [i[0] for i in words_sort]
col2 = [i[1] for i in words_sort]

with open("output.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(zip(col1, col2))


# words_sort